In [71]:
import pandas as pd

import random

In [72]:
logs = pd.read_csv('data/train_logs.csv')
logs.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1


In [73]:
ids_scores = pd.read_csv('data/train_scores.csv')
ids_scores.head()

,id,score
0,001519c8,3.5
1,0022f953,3.5
2,0042269b,6.0
3,0059420b,2.0
4,0075873a,4.0


In [74]:
# sample 80% of the data for training
ids_train = ids_scores.sample(frac=0.8, random_state=432)
ids_train.shape

(1977, 2)

In [75]:
# the rest 20% for testing
ids_test = ids_scores.drop(ids_train.index)
ids_test.shape

(494, 2)

In [76]:
# In the logs df go and replace "Move From *" with "Move From
logs['activity'] = logs['activity'].replace('Move From .*', 'Move From', regex=True)
set(logs['activity'])

{'Input', 'Move From', 'Nonproduction', 'Paste', 'Remove/Cut', 'Replace'}

In [77]:
df_group = pd.merge(logs, ids_scores, on = "id", how = "inner")
df_group.head()

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count,score
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0,3.5
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0,3.5
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1,3.5
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1,3.5


In [78]:
df_agg_new = df_group.groupby(['id']).agg(
    #event_id_max=pd.NamedAgg(column="event_id", aggfunc="max"),
    action_time_mean=pd.NamedAgg(column="action_time", aggfunc="mean"),
    action_time_sum=pd.NamedAgg(column="action_time", aggfunc="sum"),
    action_time_min=pd.NamedAgg(column="action_time", aggfunc="min"),
    action_time_max=pd.NamedAgg(column="action_time", aggfunc="max"),
    word_count_max=pd.NamedAgg(column="word_count", aggfunc="max"),
    cursor_position_max=pd.NamedAgg(column="cursor_position", aggfunc="max"),
    activity_count=pd.NamedAgg(column="activity", aggfunc="count"),
    text_change_counts = pd.NamedAgg(column="text_change", aggfunc="count"),
    down_event_counts = pd.NamedAgg(column="down_event", aggfunc="count"),
    up_event_counts = pd.NamedAgg(column="up_event", aggfunc="count"),
    score = pd.NamedAgg(column="score", aggfunc="mean")
).reset_index()
df_agg_new.head()

,id,action_time_mean,action_time_sum,action_time_min,action_time_max,word_count_max,cursor_position_max,activity_count,text_change_counts,down_event_counts,up_event_counts,score
0,001519c8,116.246774,297243,0,2259,256,1539,2557,2557,2557,2557,3.5
1,0022f953,112.221271,275391,0,1758,323,1676,2454,2454,2454,2454,3.5
2,0042269b,101.837766,421201,0,3005,404,2291,4136,4136,4136,4136,6.0
3,0059420b,121.848329,189596,0,806,206,1047,1556,1556,1556,1556,2.0
4,0075873a,123.943896,313702,0,701,252,1402,2531,2531,2531,2531,4.0


In [79]:
# Aggregate the df_group by id but each activity type is a column on it's own
df_agg = df_group.groupby(['id', 'activity']).agg(
    action_time_mean=pd.NamedAgg(column="action_time", aggfunc="mean"),
    action_time_sum=pd.NamedAgg(column="action_time", aggfunc="sum"),
    action_time_min=pd.NamedAgg(column="action_time", aggfunc="min"),
    action_time_max=pd.NamedAgg(column="action_time", aggfunc="max"),
    word_count_max=pd.NamedAgg(column="word_count", aggfunc="max"),
    cursor_position_max=pd.NamedAgg(column="cursor_position", aggfunc="max"),
    activity_count=pd.NamedAgg(column="activity", aggfunc="count"),
    text_change_counts = pd.NamedAgg(column="text_change", aggfunc="count"),
    down_event_counts = pd.NamedAgg(column="down_event", aggfunc="count"),
    up_event_counts = pd.NamedAgg(column="up_event", aggfunc="count")
).reset_index()
df_agg.head()

,id,activity,action_time_mean,action_time_sum,action_time_min,action_time_max,word_count_max,cursor_position_max,activity_count,text_change_counts,down_event_counts,up_event_counts
0,001519c8,Input,121.259204,243731,29,248,256,1495,2010,2010,2010,2010
1,001519c8,Move From,0.000000,0,0,0,256,466,3,3,3,3
2,001519c8,Nonproduction,154.216667,18506,0,2259,256,1539,120,120,120,120
3,001519c8,Remove/Cut,81.846523,34130,2,529,256,1482,417,417,417,417
4,001519c8,Replace,125.142857,876,98,196,254,1448,7,7,7,7
